In [1]:
#   |'''''''''''''╔╬╬╬╬╬╬╬╬   _____  _____      _____  _____      ___   __
#   |            ╔╬╬╬╬╬╬╬╬╬  |\   _ \  _  \    |\   _ \  _  \    |\  \|\  \
#   | ░░         ╬╬╬╬╬╬╬╬╬╬  \ \  \\__\ \  \   \ \  \\__\ \  \   \ \  \/  /|_
#    ░░░░        ╬╬╬╬╬╬╬╬╬╬   \ \  \|__| \  \   \ \  \|__| \  \   \ \   ___  \
#   ░░░░░╦╬╦    ╔╬╬╬╬╬╬╬╬╬╬    \ \  \   \ \  \   \ \  \   \ \  \   \ \  \ \   \
#  ░░░░░╬╬╬╬ ▓▓└╬╬╬╬╬╬╬╬╬╬╬     \ \__\   \ \__\   \ \__\   \ \__\   \ \__\ \___\
# ░░░░░╔╬╬╬ ▓▓▓  ╓╬╬╬╬╬╬╬╬╬      \|__|    \|__|    \|__|    \|__|    \|__| \|__|
# ░░░░░╠╬╬╬ ▓▓▓  └╬╬╬╬╬╬╬╬╬
#  ░░░░└╬╬╬╬ ▓▓   ╬╬╬╬╬╬╬╬╬  Lehrstuhl für Mensch-Maschine-Kommunikation
#  ░░░░░╙╬╬╬╩            ╬╬  Technische Universität München
#   ░░░░░░╚ '''''''''''''''  Author: Tobias Watzel
#    ░░░                     Copyright 2020
#

%matplotlib widget
import ipywidgets as widgets
import numpy as np
import dill
from IPython.display import Image
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

# init fig for matplotlib
fig, ax = None, None

## Unten werden verschiedene Klassen beschrieben. Wählen Sie eine Klasse aus und markieren Sie die Muster, die zur Klasse gehören.

In [2]:
select = widgets.Dropdown(
    options=['Summe von W1 und W2 größer als 8', 'W1 ist größer als W2', 'Das Produkt von W1 und W2 ist durch 3 teilbar', 'W1 und W2 sind größer als 3'],
    value='Summe von W1 und W2 größer als 8',
    description='Klasse auswählen:',
    layout={'width': '450px'},
    style={'description_width': '130px'},
    disabled=False,
)

box_layout = widgets.Layout(#display='flex',
                    #flex_flow='column',
                    position='relative',
                    #align_items='stretch',
                    height='32px',
                    width='250px'
                    )
bigbox_layout = widgets.Layout(#display='flex',
                    flex_flow='column',
                    position='relative',
                    #align_items='stretch',
                    height='250px',
                    width='250px'
                    )

# init state dict for saving status
save_string = 'save/state_dict_1.dill'
state_dict = {}

# try to load state_dict
try: 
    with open(save_string, 'rb') as fp:
        state_dict = dill.load(fp)
except:
    # init otherwise
    for option in select.options:
        state_dict[option] = np.zeros([6, 6])

# init variables
x, y = state_dict[list(state_dict.keys())[0]].shape
button_dict = {}
items_dict = {}
box_list = []

# item dict
for i in range(x-1, -1, -1):
    items_dict['item%i' % i] = [widgets.Label(value="W2:%i" % (i+1), layout={'width': '35px'})]
    for j in range(y):
        button_dict['button_%i_%i' % (i, j)] = widgets.Button(
            button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
            layout={'width': '30px', 'height': '30px', 'display' : '%i_%i' % (i, j)},
            discription = '%i,%i' % (i, j))
        
        # init button
        if state_dict[select.value][i, j]:
            button_dict['button_%i_%i' % (i, j)].button_style = 'success'
        else:
            button_dict['button_%i_%i' % (i, j)].button_style = 'warning'
        
        items_dict['item%i' % i].append(button_dict['button_%i_%i' % (i, j)])
    box_list.append(widgets.Box(children=items_dict['item%i' % i], layout=box_layout))    

# on_button_click function
def on_button_clicked(info):
    id = info.layout.display
    index = id.split('_')
    if info.button_style == 'success':
        info.button_style = 'warning'
        state_dict[select.value][int(index[0]), int(index[1])] = 0 # -1 because the array index start with 0 
    else:
        info.button_style = 'success'
        state_dict[select.value][int(index[0]), int(index[1])] = 1
    
    # save when changed
    with open(save_string, 'wb') as fp:
        dill.dump(state_dict, fp)
        
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        for k in range(x):
            for l in range(y):
                if state_dict[change['new']][k, l]:
                    button_dict['button_%i_%i' % (k, l)].button_style = 'success'
                else:
                    button_dict['button_%i_%i' % (k, l)].button_style = 'warning'
        
# create x axis
x_axis = [widgets.Label(value="W1:%i" % i, layout={'width': '35px'}) for i in range(1, 7)]
x_axis.insert(0, widgets.Label(value="", layout={'width': '35px'}))

box_axis = widgets.Box(children=x_axis, layout=box_layout)

# add axis to the overall box
box_list.append(box_axis)

# create widget box
box = widgets.Box(children=box_list, layout=bigbox_layout)

# display everything
display(select)
display(box)

# add on_button_clicked function
for name, item in items_dict.items():
    for box in item:
        try:
            box.on_click(on_button_clicked)
        except:
            pass
            # print('this is not a button but a label')

select.observe(on_change)

Dropdown(description='Klasse auswählen:', layout=Layout(width='450px'), options=('Summe von W1 und W2 größer a…

Box(children=(Box(children=(Label(value='W2:6', layout=Layout(width='35px')), Button(button_style='warning', l…

In [3]:
# Autograding 'Summe von W1 und W2 größer als 8', please ignore


In [4]:
# Autograding 'W1 ist größer als W2', please ignore


In [5]:
# Autograding 'Das Produkt von W1 und W2 ist durch 3 teilbar', please ignore


In [6]:
# Autograding 'W1 und W2 sind größer als 3', please ignore


In [12]:
# close old figure if avaible
if fig:
    plt.close()
# check if patterns are selected
assert np.sum(state_dict[select.value]) != 0, "Bitte mindestens ein Muster oben auswählen"
# get 1 = red coordinates from state_dict
r_x, r_y = np.where(state_dict[select.value] == 1)
# matrix to coordinates transition
r_x += 1 
r_y += 1
red = list(zip(r_x, r_y))

# get 0 = blue coordinates from state_dict
b_x, b_y = np.where(state_dict[select.value] == 0)
# matrix to coordinates transition
b_x += 1 
b_y += 1
blue = list(zip(b_x, b_y))

# define figure
fig = plt.figure(figsize=(8, 6))
ax1 = fig.add_subplot(111)

# plot everything
#plt.rcParams["figure.figsize"] = (8, 6)
ax1.plot(r_x, r_y, 'ro', color = 'r', markersize = 15)
ax1.plot(b_x, b_y, 'ro', color = 'b', markersize = 15)
ax1.set_aspect('equal')
#plt.gca().set_aspect('equal')
ax1.axis([0, 13, 0, 7])
plt.ion()
plt.tight_layout()

# create class mean red
r_x_mean, r_y_mean = np.mean(red, axis=0)
ax1.plot(r_x_mean, r_y_mean, 'x', color = 'r', markersize = 35)
# create class mean blue
b_x_mean, b_y_mean = np.mean(blue, axis=0)
ax1.plot(b_x_mean, b_y_mean, 'x', color = 'b', markersize = 35)

# calculate slope and b
m = (r_x_mean - b_x_mean) / (b_y_mean - r_y_mean)
b = (r_y_mean + b_y_mean) / 2 - m * (r_x_mean + b_x_mean) / 2

# create separation line
x_line = np.linspace(0, 7)
y_line = m * x_line + b
ax1.plot(x_line, y_line, color = 'k')

# check reclassification  
check_red = r_y - m * r_x - b
check_blue = b_y - m * b_x - b

# calculate error of reclassification
false_pos_rec = np.where(check_blue >= 0.0)
true_neg_rec = np.where(check_blue < 0.0)
false_neg_rec = np.where(check_red < 0.0)
true_pos_rec = np.where(check_red >= 0.0)

# plot wrongly classified samples 
ax1.plot(b_x[false_pos_rec], b_y[false_pos_rec], 'bo', color = 'b', markersize = 15, markeredgecolor='red')
ax1.plot(r_x[false_neg_rec], r_y[false_neg_rec], 'ro', color = 'r', markersize = 15, markeredgecolor='blue')

# calculate right recognition, false positives and false negatives
right_rec = state_dict[select.value].size - len(false_pos_rec[0]) - len(false_neg_rec[0])

# plot text
font = {'family': 'serif',
        'color':  'black',
        'weight': 'normal',
        'size': 16,
        }

plt.xticks(np.arange(0, 7, 1.0))
ax1.text(8, 6, 'True positives: %d' % len(true_pos_rec[0]), fontdict=font)
ax1.text(8, 5.5, 'True negatives: %d' % len(true_neg_rec[0]), fontdict=font)
ax1.text(8, 5, 'False positives: %d' % len(false_pos_rec[0]), fontdict=font)
ax1.text(8, 4.5, 'False negatives: %d' % len(false_neg_rec[0]), fontdict=font)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(8, 4.5, 'False negatives: 0')

##  1. Wählen Sie die Klasse "Das Produkt von W1 und W2 ist durch 3 teilbar". 
Es findet eine automatisierte Nearest-Mean-Klassifikation statt. Bestimmen Sie den Precision-Wert.

In [8]:
save_precision = 'save/precision.dill'

precision = ''
# try to load precicion value
try: 
    with open(save_precision, 'rb') as fp:
        precision = dill.load(fp)
        if precision == '':
            precision = 0.0
except:
    pass

text_precision = widgets.Text(value=str(precision),
                              placeholder='',
                              description='',
                              disabled=False,
                              continuous_update=True)

def callback_precision(change):
    if change['type'] == 'change' and change['name'] == 'value':
        # save when press enter
        with open(save_precision, 'wb') as fp:
            dill.dump(change['new'], fp)

display(text_precision)

text_precision.observe(callback_precision)

Text(value='0.5', placeholder='')

In [9]:
# Autograding Precision-Wert


Trennen Sie die Nachkommastellen bitte durch einen Punkt ab und runden Sie auf zwei Kommastellen genau.) 

## 2. Bestimmen Sie den Recall-Wert. (Gleiche Klasse wie oben.)

In [10]:
save_recall = 'save/recall.dill'

recall = ''
# try to load precicion value
try: 
    with open(save_recall, 'rb') as fp:
        recall = dill.load(fp)
        if recall == '':
            recall = 0.0
except:
    pass

text_recall = widgets.Text(value=str(recall),
                           placeholder='',
                           description='',
                           disabled=False,
                           continuous_update=True)

display(text_recall)

def callback_recall(change):
    if change['type'] == 'change' and change['name'] == 'value':
        # save when press enter
        with open(save_recall, 'wb') as fp:
            dill.dump(change['new'], fp)

text_recall.observe(callback_recall)

Text(value='0.3', placeholder='')

Trennen Sie die Nachkommastellen bitte durch einen Punkt ab und runden Sie auf zwei Kommastellen genau.) 

In [11]:
# Autograding Recall-Wert
